In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Autism_spectrum_disorder_(ASD)/GSE123302'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"A meta-analysis of two high-risk prospective studies reveals autism-specific transcriptional changes to chromatin, autoimmune, and environmental response genes in umbilical cord blood"
!Series_summary	"Autism spectrum disorder (ASD) is a neurodevelopmental disorder that affects more than 1% of children in the USA. ASD risk is thought to arise from both genetic and environmental factors, with the perinatal period as a critical window. Understanding early transcriptional changes in ASD would assist in clarifying disease pathogenesis and identifying biomarkers. However, little is known about umbilical cord blood gene expression profiles in babies later diagnosed with ASD compared to non-typically developing and non-ASD (Non-TD) or typically developing (TD) children. Genome-wide transcript levels were measured by Affymetrix Human Gene 2.0 array in RNA from cord blood samples from both the Markers of Autism Risk in Babies-Learning Early Signs (MARBLES)

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = True
trait_row = 0
age_row = None
gender_row = 1

def convert_trait(value):
    parts = value.split(':')
    if len(parts) != 2:
        return None
    diagnosis = parts[1].strip()
    if diagnosis == 'ASD':
        return 1
    elif diagnosis in ['Non-TD', 'TD']:  # Return a proper value or None if not needed
        return 0
    else:
        return None

def convert_gender(value):
    parts = value.split(':')
    if len(parts) != 2:
        return None
    gender = parts[1].strip().lower()
    if gender == 'female':
        return 0
    elif gender == 'male':
        return 1
    else:
        return None

def convert_age(value):
    return None  # Placeholder since age data is not available

save_cohort_info('GSE123302', './preprocessed/Autism_spectrum_disorder_(ASD)/cohort_info.json', is_gene_available, trait_row is not None)

if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Autism_spectrum_disorder_(ASD)', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Autism_spectrum_disorder_(ASD)/trait_data/GSE123302.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM3499537': [1, 0], 'GSM3499538': [1, 0], 'GSM3499539': [1, 0], 'GSM3499540': [1, 0], 'GSM3499541': [1, 0], 'GSM3499542': [1, 0], 'GSM3499543': [1, 0], 'GSM3499544': [1, 0], 'GSM3499545': [1, 0], 'GSM3499546': [1, 0], 'GSM3499547': [1, 0], 'GSM3499548': [1, 0], 'GSM3499549': [1, 0], 'GSM3499550': [1, 0], 'GSM3499551': [1, 0], 'GSM3499552': [1, 0], 'GSM3499553': [1, 1], 'GSM3499554': [1, 1], 'GSM3499555': [1, 1], 'GSM3499556': [1, 1], 'GSM3499557': [1, 1], 'GSM3499558': [1, 1], 'GSM3499559': [1, 1], 'GSM3499560': [1, 1], 'GSM3499561': [1, 1], 'GSM3499562': [1, 1], 'GSM3499563': [1, 1], 'GSM3499564': [1, 1], 'GSM3499565': [1, 1], 'GSM3499566': [1, 1], 'GSM3499567': [1, 1], 'GSM3499568': [1, 1], 'GSM3499569': [1, 1], 'GSM3499570': [1, 1], 'GSM3499571': [1, 1], 'GSM3499572': [1, 1], 'GSM3499573': [1, 1], 'GSM3499574': [1, 1], 'GSM3499575': [1, 1], 'GSM3499576': [1, 1], 'GSM3499577': [1, 1], 'GSM3499578': [1, 1], 'GSM3499579': [1, 1], 'GSM3499580': [1, 1], 'GSM3499581': [1, 1], 'GSM34995

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['16657436', '16657440', '16657445', '16657447', '16657450', '16657469',
       '16657473', '16657476', '16657489', '16657492', '16657502', '16657506',
       '16657514', '16657529', '16657534', '16657554', '16657572', '16657594',
       '16657598', '16657647'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['16657436', '16657440', '16657445', '16657447', '16657450'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': [12190.0, 29554.0, 69091.0, 160446.0, 317811.0], 'RANGE_END': [13639.0, 31109.0, 70008.0, 161525.0, 328581.0], 'total_probes': [25.0, 28.0, 8.0, 13.0, 36.0], 'GB_ACC': ['NR_046018', nan, nan, nan, 'NR_024368'], 'SPOT_ID': ['chr1:12190-13639', 'chr1:29554-31109', 'chr1:69091-70008', 'chr1:160446-161525', 'chr1:317811-328581'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10']}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the keys for mapping
identifier_key = 'ID'
gene_symbol_key = 'GB_ACC'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, prob_col=identifier_key, gene_col=gene_symbol_key)

# 3. Apply the mapping to get the gene expression dataframe
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Autism_spectrum_disorder_(ASD)/gene_data/GSE123302.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Autism_spectrum_disorder_(ASD)')

# 4. Save the cohort information.
save_cohort_info('GSE123302', './preprocessed/Autism_spectrum_disorder_(ASD)/cohort_info.json', True, True, trait_biased, unbiased_merged_data)

# 5. If the trait is not severely biased, save the merged data to a csv file.
if not trait_biased:
    csv_path = './preprocessed/Autism_spectrum_disorder_(ASD)/GSE123302.csv'
    unbiased_merged_data.to_csv(csv_path)


No gene data in the dataframe
